In [1]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="3"

# CycleGAN

#### Directory structure:
- **data/** contains the datasets.
    - **data/&lt;dataset&gt;/{train_A, train_B}/** contains training images for classes A and B.
    - **data/&lt;dataset&gt;/{test_A, test_B}/** contains testing images that are not used during training. These are useful to evaluate the generalization of the model to new data.
- **images/** stores metadata and loss information of each CycleGAN run, as well as evaluation images.
    - **images/meta_data.json** contains the settings of the run.
    - **images/loss_output.csv** contains the various losses of the model, stored after every batch.
    - **images/{train_A, train_B, test_A, test_B}** contains intermediate evaluation images for each epoch, illustrating generator performance.
    - **images/tmp.png** shows example image translations from the current moment in training. This image updates in real time and can be used to see how the training converges.
- **saved_models** stores the generator and discriminator models resulting from each run, which are saved every 20 epochs.

Tomado de:
https://github.com/brainhack101/IntroDL/blob/master/notebooks/2019/Eklund/CycleGAN.ipynb

!rm -rf __pycache__

!wget -nc https://raw.githubusercontent.com/brainhack101/IntroDL/master/notebooks/2019/Eklund/requirements-gpu.txt

!pip install -r requirements-gpu.txt

In [2]:
from keras.layers import Layer, Input, Dropout, Conv2D, Activation, add, UpSampling2D, Conv2DTranspose, Flatten
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization, InputSpec
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.core import Dense
from keras.optimizers import Adam
from keras.models import Model
from keras.engine.topology import Network

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from progress.bar import Bar
import datetime
import time
import json
import csv
import sys
import os

import keras.backend as K
import tensorflow as tf
print(tf.__version__)

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

1.14.0


Additional functions are contained in the `helper_functions.py` file. These mostly include code for loading the data and saving the resutls.

!wget https://raw.githubusercontent.com/brainhack101/IntroDL/master/notebooks/2019/Eklund/helper_funcs.py

In [3]:
from helper_funcs import *

If you have multiple GPUs you can select a single one of them by setting the visible CUDA device to 0, 1, ...

#### Load data

The dataset used for the run is **data/&lt;`image_folder`&gt;**.

In [4]:
image_folder = 'brain'
data = load_data(subfolder=image_folder)

### Model parameters

This CycleGAN implementation allows a lot of freedom on both the training parameters and the network architecture.

In [5]:
opt = {}

# Data
opt['channels'] = data["nr_of_channels"]
opt['img_shape'] = data["image_size"] + (opt['channels'],)
print('Image shape: ', opt['img_shape'])

opt['A_train'] = data["trainA_images"]
opt['B_train'] = data["trainB_images"]
opt['A_test'] = data["testA_images"]
opt['B_test'] = data["testB_images"]
opt['testA_image_names'] = data["testA_image_names"]
opt['testB_image_names'] = data["testB_image_names"]

opt['trainA_image_names'] = data["trainA_image_names"]
opt['trainB_image_names'] = data["trainB_image_names"]

opt['CTFOSCAL_images'] = data["CTFOSCAL_images"]
opt['CTFOSCAL_image_names'] = data["CTFOSCAL_image_names"]
opt['CTHEALTHY_images'] = data["CTHEALTHY_images"]
opt['CTHEALTHY_image_names'] = data["CTHEALTHY_image_names"]

print(opt['A_train'].shape)
print(opt['B_train'].shape)

print(opt['CTFOSCAL_images'].shape)
print(len(opt['CTFOSCAL_image_names']))
print(opt['CTHEALTHY_images'].shape)
print(len(opt['CTHEALTHY_image_names']))

Image shape:  (256, 256, 1)
(502, 256, 256, 1)
(993, 256, 256, 1)
(73, 256, 256, 1)
73
(1568, 256, 256, 1)
1568


CylceGAN can be used both on paired and unpaired data. The `paired_data` setting affects the presentation of output images as explained above.

In [6]:
#opt['paired_data'] = True
opt['paired_data'] = False

#### Training parameters
- `lambda_ABA` and `lambda_BAB` set the importance of the cycle consistency losses in relation to the adversarial loss `lambda_adversarial`
- `learning_rate_D` and `learning_rate_G` are the learning rates for the discriminators and generators respectively.
- `generator_iterations` and `discriminator_iterations` represent how many times the generators or discriminators will be trained on every batch of images. This is very useful to keep the training of both systems balanced. In this case the discriminators become successful faster than the generators, so we account for this by training the generators 3 times on every batch of images.
- `synthetic_pool_size` sets the size of the image pool used for training the discriminators. The image pool has a certain probability of returning a synthetic image from previous iterations, thus forcing the discriminator to have a certain "memory". More information on this method can be found in [this paper](https://arxiv.org/abs/1612.07828).
- `beta_1` and `beta_2` are paremeters of the [Adam](https://arxiv.org/abs/1412.6980) optimizers used on the generators and discriminators.
- `batch_size` determines the number of images used for each update of the network weights. Due to the significant memory requirements of CycleGAN it is difficult to use a large batch size. For the small example dataset values between 1-30 may be possible.
- `epochs` sets the number of training epochs. Each epoch goes through all the training images once. The number of epochs necessary to train a model is therefore dependent on both the number of training images available and the batch size.

In [7]:
# Training parameters
opt['lambda_ABA'] = 10.0  # Cyclic loss weight A_2_B
opt['lambda_BAB'] = 10.0  # Cyclic loss weight B_2_A
opt['lambda_adversarial'] = 1.0  # Weight for loss from discriminator guess on synthetic images
opt['learning_rate_D'] = 2e-4
opt['learning_rate_G'] = 2e-4
opt['generator_iterations'] = 3  # Number of generator training iterations in each training loop
opt['discriminator_iterations'] = 1  # Number of discriminator training iterations in each training loop
opt['synthetic_pool_size'] = 50  # Size of image pools used for training the discriminators
opt['beta_1'] = 0.5  # Adam parameter
opt['beta_2'] = 0.999  # Adam parameter
opt['batch_size'] = 10  # Number of images per batch
opt['epochs'] = 200  # Choose multiples of 20 since the models are saved each 20th epoch

In [8]:
# Output parameters
opt['save_models'] = True  # Save or not the generator and discriminator models
opt['save_training_img'] = True  # Save or not example training results or only tmp.png
opt['save_training_img_interval'] = 1  # Number of epoch between saves of intermediate training results
opt['self.tmp_img_update_frequency'] = 3  # Number of batches between updates of tmp.png

#### Architecture parameters
- `use_instance_normalization` is supposed to allow the selection of instance normalization or batch normalization layes. At the moment only instance normalization is implemented, so this option does not do anything.
- `use_dropout` and `use_bias` allows setting droupout layers in the generators and whether to use a bias term in the various convolutional layer in the genrators and discriminators.
- `use_linear_decay` applies linear decay on the learning rates of the generators and discriminators,   `decay_epoch`
- `use_patchgan` determines whether the discriminator evaluates the "realness" of images on a patch basis or on the whole. More information on PatchGAN can be found in [this paper](https://arxiv.org/abs/1611.07004).
- `use_resize_convolution` provides two ways to perfrom the upsampling in the generator, with significant differences in the results. More information can be found in [this article](https://distill.pub/2016/deconv-checkerboard/). Each has its advantages, and we have managed to get successful result with both methods
- `use_discriminator sigmoid` adds a sigmoid activation at the end of the discrimintator, forcing its output to the (0-1) range.

In [9]:
# Architecture parameters
opt['use_instance_normalization'] = True  # Use instance normalization or batch normalization
opt['use_dropout'] = False  # Dropout in residual blocks
opt['use_bias'] = True  # Use bias
opt['use_linear_decay'] = True  # Linear decay of learning rate, for both discriminators and generators
opt['decay_epoch'] = 100  # The epoch where the linear decay of the learning rates start
opt['use_patchgan'] = True  # PatchGAN - if false the discriminator learning rate should be decreased
opt['use_resize_convolution'] = False  # Resize convolution - instead of transpose convolution in deconvolution layers (uk) - can reduce checkerboard artifacts but the blurring might affect the cycle-consistency
opt['discriminator_sigmoid'] = True  # Add a final sigmoid activation to the discriminator

In [10]:
# Tweaks
opt['REAL_LABEL'] = 1.0  # Use e.g. 0.9 to avoid training the discriminators to zero loss

### Model architecture

#### Layer blocks
These are the individual layer blocks that are used to build the generators and discriminator. More information can be found in the appendix of the [CycleGAN paper](https://arxiv.org/abs/1703.10593).

In [11]:
# Discriminator layers
def ck(model, opt, x, k, use_normalization, use_bias):
    x = Conv2D(filters=k, kernel_size=4, strides=2, padding='same', use_bias=use_bias)(x)
    if use_normalization:
        x = model['normalization'](axis=3, center=True, epsilon=1e-5)(x, training=True)
    x = LeakyReLU(alpha=0.2)(x)
    return x

# First generator layer
def c7Ak(model, opt, x, k):
    x = Conv2D(filters=k, kernel_size=7, strides=1, padding='valid', use_bias=opt['use_bias'])(x)
    x = model['normalization'](axis=3, center=True, epsilon=1e-5)(x, training=True)
    x = Activation('relu')(x)
    return x

# Downsampling
def dk(model, opt, x, k):  # Should have reflection padding
    x = Conv2D(filters=k, kernel_size=3, strides=2, padding='same', use_bias=opt['use_bias'])(x)
    x = model['normalization'](axis=3, center=True, epsilon=1e-5)(x, training=True)
    x = Activation('relu')(x)
    return x

# Residual block
def Rk(model, opt, x0):
    k = int(x0.shape[-1])

    # First layer
    x = ReflectionPadding2D((1,1))(x0)
    x = Conv2D(filters=k, kernel_size=3, strides=1, padding='valid', use_bias=opt['use_bias'])(x)
    x = model['normalization'](axis=3, center=True, epsilon=1e-5)(x, training=True)
    x = Activation('relu')(x)

    if opt['use_dropout']:
        x = Dropout(0.5)(x)

    # Second layer
    x = ReflectionPadding2D((1, 1))(x)
    x = Conv2D(filters=k, kernel_size=3, strides=1, padding='valid', use_bias=opt['use_bias'])(x)
    x = model['normalization'](axis=3, center=True, epsilon=1e-5)(x, training=True)
    # Merge
    x = add([x, x0])

    return x

# Upsampling
def uk(model, opt, x, k):
    # (up sampling followed by 1x1 convolution <=> fractional-strided 1/2)
    if opt['use_resize_convolution']:
        x = UpSampling2D(size=(2, 2))(x)  # Nearest neighbor upsampling
        x = ReflectionPadding2D((1, 1))(x)
        x = Conv2D(filters=k, kernel_size=3, strides=1, padding='valid', use_bias=opt['use_bias'])(x)
    else:
        x = Conv2DTranspose(filters=k, kernel_size=3, strides=2, padding='same', use_bias=opt['use_bias'])(x)  # this matches fractionally stided with stride 1/2
    x = model['normalization'](axis=3, center=True, epsilon=1e-5)(x, training=True)
    x = Activation('relu')(x)
    return x

#### Architecture functions

In [12]:
def build_discriminator(model, opt, name=None):
    # Input
    input_img = Input(shape=opt['img_shape'])

    # Layers 1-4
    x = ck(model, opt, input_img, 64, False, True) #  Instance normalization is not used for this layer)
    x = ck(model, opt, x, 128, True, opt['use_bias'])
    x = ck(model, opt, x, 256, True, opt['use_bias'])
    x = ck(model, opt, x, 512, True, opt['use_bias'])

    # Layer 5: Output
    if opt['use_patchgan']:
        x = Conv2D(filters=1, kernel_size=4, strides=1, padding='same', use_bias=True)(x)
    else:
        x = Flatten()(x)
        x = Dense(1)(x)

    if opt['discriminator_sigmoid']:
        x = Activation('sigmoid')(x)

    return Model(inputs=input_img, outputs=x, name=name)

def build_generator(model, opt, name=None):
    # Layer 1: Input
    input_img = Input(shape=opt['img_shape'])
    x = ReflectionPadding2D((3, 3))(input_img)
    x = c7Ak(model, opt, x, 32)

    # Layer 2-3: Downsampling
    x = dk(model, opt, x, 64)
    x = dk(model, opt, x, 128)

    # Layers 4-12: Residual blocks
    for _ in range(4, 13):
        x = Rk(model, opt, x)

    # Layer 13:14: Upsampling
    x = uk(model, opt, x, 64)
    x = uk(model, opt, x, 32)

    # Layer 15: Output
    x = ReflectionPadding2D((3, 3))(x)
    x = Conv2D(opt['channels'], kernel_size=7, strides=1, padding='valid', use_bias=True)(x)
    x = Activation('tanh')(x)

    return Model(inputs=input_img, outputs=x, name=name)

#### CasNet generator

In [13]:
#@title Generator
from keras import layers

def u_block(input_tensor, 
            encoder_num_filters=[64, 128, 256, 512, 512, 512, 512, 512],
            decoder_num_filters=[512, 1024, 1024, 1024, 1024, 512, 256, 128],
            use_instance_norm=True,
            block_prefix=''):
    
    x = input_tensor
    encoder_blocks_outputs = []
    
    # Encoder path.
    for i, num_filters in enumerate(encoder_num_filters):
        x = layers.Conv2D(num_filters, 
                          kernel_size=4, 
                          strides=2, 
                          padding='same', 
                          name=f'{block_prefix}_EncoderBlock{i+1}-Conv')(x)
        if use_instance_norm:
            x = InstanceNormalization(name=f'{block_prefix}_EncoderBlock{i+1}-Instancenorm')(x)
        else:
            x = layers.BatchNormalization(name=f'{block_prefix}_EncoderBlock{i+1}-Batchnorm')(x)
        x = layers.Activation('relu', name=f'{block_prefix}_EncoderBlock{i+1}-ReLU')(x)
        
        # Append the encoder blocks outputs, except for the last one.
        if i != len(encoder_num_filters) - 1: 
            encoder_blocks_outputs.append(x)

    # Decoder path.
    for i, num_filters in enumerate(decoder_num_filters):
        x = layers.Conv2DTranspose(
            num_filters, 
            kernel_size=(4, 4), 
            strides=(2, 2), 
            padding='same',
            name=f'{block_prefix}_DecoderBlock{i+1}-TransposedConv'
        )(x)
        x = layers.Activation('relu', name=f'{block_prefix}_DecoderBlock{i+1}-ReLU')(x)

        # All the decoder blocks have concatenate the encoder block output with
        # the same spatial dimentions except for the last one.
        if i != len(decoder_num_filters) - 1: 
            x = layers.Concatenate(
                name=f'{block_prefix}_DecoderBlock{i+1}-Concat'
            )([encoder_blocks_outputs[-(i + 1)], x])

    # Last operation: 1x1 conv to map the image to the input n_channels.
    x = layers.Conv2D(1,   #input_tensor.shape[-1]
                      kernel_size=(1, 1), 
                      name=f'{block_prefix}_Decoder-Conv1x1')(x)
    output = layers.Activation('tanh', name=f'{block_prefix}_Decoder-Tanh')(x)

    return output

def build_casnet_generator(input_shape=(256, 256, 1), 
                           n_blocks=2, 
                           use_instance_norm=True,
                           name=None):
    """Create a CasNet Generator using UBlocks"""
    input_tensor = Input(shape=input_shape)
    x = input_tensor
    for i in range(n_blocks):
        x = u_block(x, block_prefix=f'UBlock{i+1}', use_instance_norm=use_instance_norm)
    
    return Model(inputs=input_tensor, outputs=x, name=name)

#### Loss functions
The discriminators use MSE loss. The generators use MSE for the adversarial losses and MAE for the cycle consistency losses.

In [14]:
# Mean squared error
def mse(y_true, y_pred):
    loss = tf.reduce_mean(tf.squared_difference(y_pred, y_true))
    return loss

# Mean absolute error
def mae(y_true, y_pred):
    loss = tf.reduce_mean(tf.abs(y_pred - y_true))
    return loss

#### Build CycleGAN model

In [15]:
model = {}

# Normalization
model['normalization'] = InstanceNormalization

# Optimizers
model['opt_D'] = Adam(opt['learning_rate_D'], opt['beta_1'], opt['beta_2'])
model['opt_G'] = Adam(opt['learning_rate_G'], opt['beta_1'], opt['beta_2'])

# Build discriminators
D_A = build_discriminator(model, opt, name='D_A')
D_B = build_discriminator(model, opt, name='D_B')

# Define discriminator models
image_A = Input(shape=opt['img_shape'])
print(image_A.shape)
image_B = Input(shape=opt['img_shape'])
guess_A = D_A(image_A)
guess_B = D_B(image_B)
model['D_A'] = Model(inputs=image_A, outputs=guess_A, name='D_A_model')
model['D_B'] = Model(inputs=image_B, outputs=guess_B, name='D_B_model')

# Compile discriminator models
loss_weights_D = [0.5]  # 0.5 since we train on real and synthetic images
model['D_A'].compile(optimizer=model['opt_D'],
                 loss=mse,
                 loss_weights=loss_weights_D)
model['D_B'].compile(optimizer=model['opt_D'],
                 loss=mse,
                 loss_weights=loss_weights_D)

# Use containers to make a static copy of discriminators, used when training the generators
model['D_A_static'] = Network(inputs=image_A, outputs=guess_A, name='D_A_static_model')
model['D_B_static'] = Network(inputs=image_B, outputs=guess_B, name='D_B_static_model')

# Do not update discriminator weights during generator training
model['D_A_static'].trainable = False
model['D_B_static'].trainable = False

# Build generators
model['G_A2B'] = build_generator(model, opt, name='G_A2B_model')
model['G_B2A'] = build_generator(model, opt, name='G_B2A_model')

#model['G_A2B'] = build_casnet_generator(input_shape=(256,256,1), n_blocks=1, name='G_A2B_model')
#model['G_B2A'] = build_casnet_generator(input_shape=(256,256,1), n_blocks=1, name='G_B2A_model')

print("G_A2B")
model['G_A2B'].summary()
#print("G_B2A")
#model['G_B2A'].summary()

# Define full CycleGAN model, used for training the generators
real_A = Input(shape=opt['img_shape'], name='real_A')
real_B = Input(shape=opt['img_shape'], name='real_B')
synthetic_B = model['G_A2B'](real_A)
synthetic_A = model['G_B2A'](real_B)
dB_guess_synthetic = model['D_B_static'](synthetic_B)
dA_guess_synthetic = model['D_A_static'](synthetic_A)
reconstructed_A = model['G_B2A'](synthetic_B)
reconstructed_B = model['G_A2B'](synthetic_A)

# Compile full CycleGAN model
model_outputs = [reconstructed_A, reconstructed_B,
                 dB_guess_synthetic, dA_guess_synthetic]
compile_losses = [mae, mae,
                  mse, mse]
compile_weights = [opt['lambda_ABA'], opt['lambda_BAB'],
                   opt['lambda_adversarial'], opt['lambda_adversarial']]

model['G_model'] = Model(inputs=[real_A, real_B],
                     outputs=model_outputs,
                     name='G_model')

model['G_model'].compile(optimizer=model['opt_G'],
                     loss=compile_losses,
                     loss_weights=compile_weights)

W0125 16:57:32.302791 140269506586432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:74: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0125 16:57:32.304666 140269506586432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0125 16:57:32.308792 140269506586432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0125 16:57:32.665253 140269506586432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/optimizers.py:790: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.



(?, 256, 256, 1)
G_A2B
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            (None, 256, 256, 1)  0                                            
__________________________________________________________________________________________________
reflection_padding2d_1 (Reflect (None, 262, 262, 1)  0           input_5[0][0]                    
__________________________________________________________________________________________________
conv2d_11 (Conv2D)              (None, 256, 256, 32) 1600        reflection_padding2d_1[0][0]     
__________________________________________________________________________________________________
instance_normalization_7 (Insta (None, 256, 256, 32) 64          conv2d_11[0][0]                  
______________________________________________________________________________________

#### Folders and configuration

In [16]:
opt['date_time'] = time.strftime('%Y%m%d-%H%M%S', time.localtime()) + '-' + image_folder

# Output folder for run data and images
opt['out_dir'] = os.path.join('images', opt['date_time'])
if not os.path.exists(opt['out_dir']):
    os.makedirs(opt['out_dir'])

# Output folder for saved models
if opt['save_models']:
    opt['model_out_dir'] = os.path.join('saved_models', opt['date_time'])
    if not os.path.exists(opt['model_out_dir']):
        os.makedirs(opt['model_out_dir'])

write_metadata_to_JSON(model, opt)

# Don't pre-allocate GPU memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
K.tensorflow_backend.set_session(tf.Session(config=config))

### Training function

In [17]:
def train(model, opt):

    def run_training_batch():

        # ======= Discriminator training ======
        # Generate batch of synthetic images
        synthetic_images_B = model['G_A2B'].predict(real_images_A)
        synthetic_images_A = model['G_B2A'].predict(real_images_B)
        synthetic_images_B = synthetic_pool_B.query(synthetic_images_B)
        synthetic_images_A = synthetic_pool_A.query(synthetic_images_A)

        # Train discriminators on batch
        D_loss = []
        for _ in range(opt['discriminator_iterations']):
            D_A_loss_real = model['D_A'].train_on_batch(x=real_images_A, y=ones)
            D_B_loss_real = model['D_B'].train_on_batch(x=real_images_B, y=ones)
            D_A_loss_synthetic = model['D_A'].train_on_batch(x=synthetic_images_A, y=zeros)
            D_B_loss_synthetic = model['D_B'].train_on_batch(x=synthetic_images_B, y=zeros)
            D_A_loss = D_A_loss_real + D_A_loss_synthetic
            D_B_loss = D_B_loss_real + D_B_loss_synthetic
            D_loss.append(D_A_loss + D_B_loss)

        # ======= Generator training ==========
        target_data = [real_images_A, real_images_B, ones, ones]  # Reconstructed images need to match originals, discriminators need to predict ones

        # Train generators on batch
        G_loss = []
        for _ in range(opt['generator_iterations']):
            G_loss.append(model['G_model'].train_on_batch(
                x=[real_images_A, real_images_B], y=target_data))

        # =====================================

        # Update learning rates
        if opt['use_linear_decay'] and epoch >= opt['decay_epoch']:
            update_lr(model['D_A'], decay_D)
            update_lr(model['D_B'], decay_D)
            update_lr(model['G_model'], decay_G)

        # Store training losses
        D_A_losses.append(D_A_loss)
        D_B_losses.append(D_B_loss)
        D_losses.append(D_loss[-1])

        ABA_reconstruction_loss = G_loss[-1][1]
        BAB_reconstruction_loss = G_loss[-1][2]
        reconstruction_loss = ABA_reconstruction_loss + BAB_reconstruction_loss
        G_AB_adversarial_loss = G_loss[-1][3]
        G_BA_adversarial_loss = G_loss[-1][4]

        ABA_reconstruction_losses.append(ABA_reconstruction_loss)
        BAB_reconstruction_losses.append(BAB_reconstruction_loss)
        reconstruction_losses.append(reconstruction_loss)
        G_AB_adversarial_losses.append(G_AB_adversarial_loss)
        G_BA_adversarial_losses.append(G_BA_adversarial_loss)
        G_losses.append(G_loss[-1][0])

        # Print training status
        print('\n')
        print('Epoch ---------------------', epoch, '/', opt['epochs'])
        print('Loop index ----------------', loop_index + 1, '/', nr_im_per_epoch)
        if opt['discriminator_iterations'] > 1:
            print('  Discriminator losses:')
            for i in range(opt['discriminator_iterations']):
                print('D_loss', D_loss[i])
        if opt['generator_iterations'] > 1:
            print('  Generator losses:')
            for i in range(opt['generator_iterations']):
                print('G_loss', G_loss[i])
        print('  Summary:')
        print('D_lr:', K.get_value(model['D_A'].optimizer.lr))
        print('G_lr', K.get_value(model['G_model'].optimizer.lr))
        print('D_loss: ', D_loss[-1])
        print('G_loss: ', G_loss[-1][0])
        print('reconstruction_loss: ', reconstruction_loss)
        print_ETA(opt, start_time, epoch, nr_im_per_epoch, loop_index)
        sys.stdout.flush()

        if loop_index % 3*opt['batch_size'] == 0:
            # Save temporary images continously
            save_tmp_images(model, opt, real_images_A[0], real_images_B[0],
                                 synthetic_images_A[0], synthetic_images_B[0])

    # ======================================================================
    # Begin training
    # ======================================================================
    if opt['save_training_img'] and not os.path.exists(os.path.join(opt['out_dir'], 'train_A')):
        os.makedirs(os.path.join(opt['out_dir'], 'train_A'))
        os.makedirs(os.path.join(opt['out_dir'], 'train_B'))
        os.makedirs(os.path.join(opt['out_dir'], 'test_A'))
        os.makedirs(os.path.join(opt['out_dir'], 'test_B'))

    D_A_losses = []
    D_B_losses = []
    D_losses = []

    ABA_reconstruction_losses = []
    BAB_reconstruction_losses = []
    reconstruction_losses = []
    G_AB_adversarial_losses = []
    G_BA_adversarial_losses = []
    G_losses = []

    # Image pools used to update the discriminators
    synthetic_pool_A = ImagePool(opt['synthetic_pool_size'])
    synthetic_pool_B = ImagePool(opt['synthetic_pool_size'])

    # Labels used for discriminator training
    label_shape = (opt['batch_size'],) + model['D_A'].output_shape[1:]
    ones = np.ones(shape=label_shape) * opt['REAL_LABEL']
    zeros = ones * 0

    # Linear learning rate decay
    if opt['use_linear_decay']:
        decay_D, decay_G = get_lr_linear_decay_rate(opt)

    nr_train_im_A = opt['A_train'].shape[0]
    nr_train_im_B = opt['B_train'].shape[0]
    nr_im_per_epoch = int(np.ceil(np.max((nr_train_im_A, nr_train_im_B)) / opt['batch_size']) * opt['batch_size'])

    # Start stopwatch for ETAs
    start_time = time.time()
    timer_started = False

    for epoch in range(1, opt['epochs'] + 1):
        # random_order_A = np.random.randint(nr_train_im_A, size=nr_im_per_epoch)
        # random_order_B = np.random.randint(nr_train_im_B, size=nr_im_per_epoch)

        random_order_A = np.concatenate((np.random.permutation(nr_train_im_A),
                                         np.random.randint(nr_train_im_A, size=nr_im_per_epoch - nr_train_im_A)))
        random_order_B = np.concatenate((np.random.permutation(nr_train_im_B),
                                         np.random.randint(nr_train_im_B, size=nr_im_per_epoch - nr_train_im_B)))

        # Train on image batch
        for loop_index in range(0, nr_im_per_epoch, opt['batch_size']):
            indices_A = random_order_A[loop_index:loop_index + opt['batch_size']]
            indices_B = random_order_B[loop_index:loop_index + opt['batch_size']]

            real_images_A = opt['A_train'][indices_A]
            real_images_B = opt['B_train'][indices_B]

            # Train on image batch
            run_training_batch()

            # Start timer after first (slow) iteration has finished
            if not timer_started:
                start_time = time.time()
                timer_started = True

        # Save training images
        if opt['save_training_img'] and epoch % opt['save_training_img_interval'] == 0:
            print('\n', '\n', '-------------------------Saving images for epoch', epoch, '-------------------------', '\n', '\n')
            save_epoch_images(model, opt, epoch)

        # Save model
        if opt['save_models'] and epoch % 10 == 0:
            save_model(opt, model['D_A'], epoch)
            save_model(opt, model['D_B'], epoch)
            save_model(opt, model['G_A2B'], epoch)
            save_model(opt, model['G_B2A'], epoch)

        # Save training history
        training_history = {
            'DA_losses': D_A_losses,
            'DB_losses': D_B_losses,
            'G_AB_adversarial_losses': G_AB_adversarial_losses,
            'G_BA_adversarial_losses': G_BA_adversarial_losses,
            'ABA_reconstruction_losses': ABA_reconstruction_losses,
            'BAB_reconstruction_losses': BAB_reconstruction_losses,
            'reconstruction_losses': reconstruction_losses,
            'D_losses': D_losses,
            'G_losses': G_losses}
        write_loss_data_to_file(opt, training_history)

### Train CycleGAN

In [ ]:
train(model, opt)

W0125 16:57:37.149694 140269506586432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:2741: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.

W0125 16:57:37.263936 140269506586432 deprecation_wrapper.py:119] From /usr/local/lib/python3.6/dist-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.





Epoch --------------------- 1 / 200
Loop index ---------------- 1 / 1000
  Generator losses:
G_loss [18.590385, 0.87451154, 0.8996269, 0.3950256, 0.45397514]
G_loss [15.556129, 0.74503744, 0.7448535, 0.31073502, 0.3464847]
G_loss [13.2519455, 0.6394499, 0.62756395, 0.30182642, 0.27998132]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.6567475
G_loss:  13.2519455
reconstruction_loss:  1.2670138
Elapsed time 0:00:47 : ETA in 1080140 days, 10:15:19


Epoch --------------------- 1 / 200
Loop index ---------------- 11 / 1000
  Generator losses:
G_loss [11.691724, 0.5540344, 0.5253623, 0.45022994, 0.44752663]
G_loss [9.883981, 0.4701209, 0.4326438, 0.4332188, 0.4231146]
G_loss [8.374229, 0.40186128, 0.3530808, 0.425486, 0.39932287]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.49824315
G_loss:  8.374229
reconstruction_loss:  0.75494206
Elapsed time 0:00:03 : ETA in 15:41:44


Epoch --------------------- 1 / 200
Loop index ---------------- 21 / 1000
  Generator losses:
G_loss [7.543932, 0.



Epoch --------------------- 1 / 200
Loop index ---------------- 191 / 1000
  Generator losses:
G_loss [2.3773394, 0.07905884, 0.07081197, 0.4999261, 0.37870526]
G_loss [2.3347025, 0.075143, 0.07155995, 0.4990911, 0.3685819]
G_loss [2.359162, 0.07527636, 0.075430535, 0.49416438, 0.35792866]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.6514622
G_loss:  2.359162
reconstruction_loss:  0.15070689
Elapsed time 0:00:56 : ETA in 16:15:10


Epoch --------------------- 1 / 200
Loop index ---------------- 201 / 1000
  Generator losses:
G_loss [2.4370847, 0.08802515, 0.05901596, 0.54730934, 0.41936415]
G_loss [2.2834475, 0.08523723, 0.048324913, 0.5368923, 0.4109338]
G_loss [2.1998024, 0.08130296, 0.0447556, 0.5267778, 0.41243887]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.54341483
G_loss:  2.1998024
reconstruction_loss:  0.12605856
Elapsed time 0:00:59 : ETA in 16:14:07


Epoch --------------------- 1 / 200
Loop index ---------------- 211 / 1000
  Generator losses:
G_loss [2.1263676, 0.06



Epoch --------------------- 1 / 200
Loop index ---------------- 381 / 1000
  Generator losses:
G_loss [2.126975, 0.08003656, 0.044599608, 0.4864975, 0.39411598]
G_loss [2.1090662, 0.06787826, 0.057370067, 0.4806052, 0.37597767]
G_loss [1.9486282, 0.059106432, 0.049462967, 0.48668808, 0.37624627]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.4806811
G_loss:  1.9486282
reconstruction_loss:  0.1085694
Elapsed time 0:01:51 : ETA in 16:13:20


Epoch --------------------- 1 / 200
Loop index ---------------- 391 / 1000
  Generator losses:
G_loss [2.0807784, 0.062191594, 0.06481926, 0.44536906, 0.36530083]
G_loss [2.0364604, 0.060641795, 0.06324466, 0.43886852, 0.35872722]
G_loss [1.9842955, 0.058426537, 0.060480256, 0.43751654, 0.35771093]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.56694317
G_loss:  1.9842955
reconstruction_loss:  0.118906796
Elapsed time 0:01:54 : ETA in 16:12:45


Epoch --------------------- 1 / 200
Loop index ---------------- 401 / 1000
  Generator losses:
G_loss [2



Epoch --------------------- 1 / 200
Loop index ---------------- 571 / 1000
  Generator losses:
G_loss [1.696024, 0.050429605, 0.033794783, 0.4189207, 0.43485922]
G_loss [1.636128, 0.048492454, 0.031040195, 0.41420054, 0.42660087]
G_loss [1.5936978, 0.047551524, 0.029818852, 0.412754, 0.40724006]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.4589465
G_loss:  1.5936978
reconstruction_loss:  0.077370375
Elapsed time 0:02:46 : ETA in 16:10:53


Epoch --------------------- 1 / 200
Loop index ---------------- 581 / 1000
  Generator losses:
G_loss [1.7276629, 0.048661906, 0.039877012, 0.43600217, 0.4062716]
G_loss [1.702738, 0.04829048, 0.038864452, 0.4384609, 0.3927278]
G_loss [1.7594544, 0.05666999, 0.03702777, 0.43298936, 0.3894874]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.484376
G_loss:  1.7594544
reconstruction_loss:  0.09369776
Elapsed time 0:02:49 : ETA in 16:11:15


Epoch --------------------- 1 / 200
Loop index ---------------- 591 / 1000
  Generator losses:
G_loss [1.814954

...



Epoch --------------------- 11 / 200
Loop index ---------------- 441 / 1000
  Generator losses:
G_loss [1.4630786, 0.030769378, 0.018069372, 0.45366365, 0.52102745]
G_loss [1.3814718, 0.029227, 0.016899712, 0.44617516, 0.47402945]
G_loss [1.2969619, 0.028558632, 0.014051336, 0.41168466, 0.45917764]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.41221052
G_loss:  1.2969619
reconstruction_loss:  0.042609967
Elapsed time 0:51:09 : ETA in 15:28:52


Epoch --------------------- 11 / 200
Loop index ---------------- 451 / 1000
  Generator losses:
G_loss [1.4753107, 0.036396854, 0.02612253, 0.39578295, 0.45433387]
G_loss [1.402746, 0.034672428, 0.026834112, 0.3818048, 0.40587583]
G_loss [1.4800148, 0.041754495, 0.02658068, 0.38578534, 0.41087776]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.48517275
G_loss:  1.4800148
reconstruction_loss:  0.068335176
Elapsed time 0:51:12 : ETA in 15:28:47


Epoch --------------------- 11 / 200
Loop index ---------------- 461 / 1000
  Generator losses:
G_l



Epoch --------------------- 11 / 200
Loop index ---------------- 631 / 1000
  Generator losses:
G_loss [1.4134316, 0.028689455, 0.022069167, 0.39299956, 0.5128459]
G_loss [1.3395797, 0.026900942, 0.021124398, 0.37819347, 0.48113284]
G_loss [1.2810026, 0.026109904, 0.019543327, 0.3699788, 0.45449153]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.37654662
G_loss:  1.2810026
reconstruction_loss:  0.04565323
Elapsed time 0:52:05 : ETA in 15:27:43


Epoch --------------------- 11 / 200
Loop index ---------------- 641 / 1000
  Generator losses:
G_loss [1.3512945, 0.02740323, 0.022552539, 0.38216496, 0.46957183]
G_loss [1.2817416, 0.025957242, 0.024032336, 0.3701735, 0.41167235]
G_loss [1.2631321, 0.025583336, 0.02676255, 0.35883635, 0.38083684]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.47879624
G_loss:  1.2631321
reconstruction_loss:  0.052345887
Elapsed time 0:52:08 : ETA in 15:27:40


Epoch --------------------- 11 / 200
Loop index ---------------- 651 / 1000
  Generator losses:
G_



Epoch --------------------- 11 / 200
Loop index ---------------- 821 / 1000
  Generator losses:
G_loss [1.4084734, 0.02301404, 0.026833331, 0.3540029, 0.5559968]
G_loss [1.2665737, 0.019527808, 0.018614361, 0.33401722, 0.55113477]
G_loss [1.2261165, 0.023640057, 0.014903136, 0.31080836, 0.52987623]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.4836185
G_loss:  1.2261165
reconstruction_loss:  0.038543195
Elapsed time 0:53:00 : ETA in 15:26:35


Epoch --------------------- 11 / 200
Loop index ---------------- 831 / 1000
  Generator losses:
G_loss [1.5084066, 0.033037685, 0.03593144, 0.30938977, 0.5093257]
G_loss [1.8451703, 0.039934747, 0.06350658, 0.32555237, 0.48520464]
G_loss [1.8169794, 0.062653266, 0.04173077, 0.30222076, 0.47091812]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.5275812
G_loss:  1.8169794
reconstruction_loss:  0.104384035
Elapsed time 0:53:03 : ETA in 15:26:31


Epoch --------------------- 11 / 200
Loop index ---------------- 841 / 1000
  Generator losses:
G_los



Epoch --------------------- 12 / 200
Loop index ---------------- 11 / 1000
  Generator losses:
G_loss [1.3701082, 0.027253404, 0.023191614, 0.37511325, 0.4905448]
G_loss [1.3098341, 0.025843527, 0.021321386, 0.36897674, 0.46920824]
G_loss [1.2764492, 0.02516199, 0.022654762, 0.35889497, 0.43938667]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.49422747
G_loss:  1.2764492
reconstruction_loss:  0.047816753
Elapsed time 0:53:55 : ETA in 15:25:34


Epoch --------------------- 12 / 200
Loop index ---------------- 21 / 1000
  Generator losses:
G_loss [1.4288623, 0.029488033, 0.015474129, 0.42402777, 0.5552129]
G_loss [1.4074552, 0.031692967, 0.015318965, 0.4191366, 0.51819927]
G_loss [1.5319321, 0.03893568, 0.019723047, 0.40963298, 0.5357118]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.3842548
G_loss:  1.5319321
reconstruction_loss:  0.058658727
Elapsed time 0:53:58 : ETA in 15:25:30


Epoch --------------------- 12 / 200
Loop index ---------------- 31 / 1000
  Generator losses:
G_loss



Epoch --------------------- 12 / 200
Loop index ---------------- 201 / 1000
  Generator losses:
G_loss [1.6626409, 0.039196648, 0.024485685, 0.36328375, 0.6625339]
G_loss [1.4942713, 0.033816047, 0.014208111, 0.36322412, 0.6508056]
G_loss [1.3782188, 0.026956925, 0.011582688, 0.35605136, 0.63677126]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.38737082
G_loss:  1.3782188
reconstruction_loss:  0.038539615
Elapsed time 0:54:50 : ETA in 15:24:24


Epoch --------------------- 12 / 200
Loop index ---------------- 211 / 1000
  Generator losses:
G_loss [1.3026977, 0.030795166, 0.02570187, 0.34709692, 0.39063042]
G_loss [1.2306598, 0.027744014, 0.025478963, 0.33808818, 0.36034188]
G_loss [1.1871313, 0.027553737, 0.024649138, 0.33213606, 0.33296642]
  Summary:
D_lr: 0.0002
G_lr 0.0002
D_loss:  0.51988083
G_loss:  1.1871313
reconstruction_loss:  0.052202873
Elapsed time 0:54:53 : ETA in 15:24:20


Epoch --------------------- 12 / 200
Loop index ---------------- 221 / 1000
  Generator losses:

### Load weights

In [ ]:
#folder = "20211209-235929-brain" #2ublocks
#folder = "20211213-234440-brain" #1ublocks
#folder = "20211213-234440-brain" #1ublocks 6epochs ---> embebidos
#folder = "20211214-223746-brain" #2ublocks 2epochs
#folder = "20211214-232746-brain" #2ublocks 10epochs ---> embebidos
#folder = "20211215-141924-brain" #2ublocks 4epochs 4g1d
#folder = "20211215-175957-brain" #2ublocks 4epochs 3g2d

folder = "20220112-215315-brain" #1ublocks 10epochs (mejor valor con 5 epochs)

#folder = "20220119-154149-brain" #BORRAR 2ublocks 10 epochs

ep = 1
checkpoint_path1 = "saved_models/"+folder+"/G_A2B_model_weights_epoch_"+str(ep)+".hdf5"
checkpoint_path2 = "saved_models/"+folder+"/G_B2A_model_weights_epoch_"+str(ep)+".hdf5"
model["G_A2B"].load_weights(checkpoint_path1)
model["G_B2A"].load_weights(checkpoint_path2)

In [ ]:
test_images_A = opt['A_train'][1]
test_images_B = opt['B_train'][1]
print(opt['A_train'].shape)
print(opt['B_train'].shape)

synthetic_images_B = model['G_A2B'].predict(test_images_A[np.newaxis,:])
reconstructed_image_A = model['G_B2A'].predict(synthetic_images_B)

synthetic_images_A = model['G_B2A'].predict(test_images_B[np.newaxis,:])
reconstructed_image_B = model['G_A2B'].predict(synthetic_images_A)

plt.figure(figsize=(15,20))
plt.subplot(4,3,1)
plt.imshow(test_images_A[:,:,0], cmap="gray")
plt.subplot(4,3,2)
plt.imshow(synthetic_images_B[0,:,:,0], cmap="gray")
plt.subplot(4,3,3)
plt.imshow(reconstructed_image_A[0,:,:,0], cmap="gray")

plt.subplot(4,3,4)
plt.imshow(test_images_B[:,:,0], cmap="gray")
plt.subplot(4,3,5)
plt.imshow(synthetic_images_A[0,:,:,0], cmap="gray")
plt.subplot(4,3,6)
plt.imshow(reconstructed_image_B[0,:,:,0], cmap="gray")


CTHEALTHY_images_A = opt['CTHEALTHY_images'][1]

synthetic_images_B = model['G_A2B'].predict(CTHEALTHY_images_A[np.newaxis,:])
reconstructed_image_A = model['G_B2A'].predict(synthetic_images_B)

plt.subplot(4,3,7)
plt.imshow(CTHEALTHY_images_A[:,:,0], cmap="gray")
plt.subplot(4,3,8)
plt.imshow(synthetic_images_B[0,:,:,0], cmap="gray")
plt.subplot(4,3,9)
plt.imshow(reconstructed_image_A[0,:,:,0], cmap="gray")


CTFOSCAL_images_A = opt['CTFOSCAL_images'][10]

synthetic_images_B = model['G_A2B'].predict(CTFOSCAL_images_A[np.newaxis,:])
reconstructed_image_A = model['G_B2A'].predict(synthetic_images_B)

plt.subplot(4,3,10)
plt.imshow(CTFOSCAL_images_A[:,:,0], cmap="gray")
plt.subplot(4,3,11)
plt.imshow(synthetic_images_B[0,:,:,0], cmap="gray")
plt.subplot(4,3,12)
plt.imshow(reconstructed_image_A[0,:,:,0], cmap="gray")
plt.show()

# Training data

# Synthetic MRI

In [ ]:
import keras

import seaborn as sns

capa_o = keras.models.Model(inputs=model["G_A2B"].get_input_at(0), outputs=model["G_A2B"].get_layer('UBlock1_EncoderBlock8-ReLU').output)
capa_o2 = keras.models.Model(inputs=model["G_B2A"].get_input_at(0), outputs=model["G_B2A"].get_layer('UBlock1_EncoderBlock8-ReLU').output)

In [ ]:
synthetic_images_B = model['G_A2B'].predict(opt['A_train'])

synthetic_images_B_CTHEALTHY = model['G_A2B'].predict(opt['CTHEALTHY_images'])
synthetic_images_B_CTFOSCAL = model['G_A2B'].predict(opt['CTFOSCAL_images'])

print(opt['trainA_image_names'][0])
print(opt['CTHEALTHY_image_names'][0])
print(opt['CTFOSCAL_image_names'][0])

emb = capa_o.predict(synthetic_images_B)
print(emb[:,0,0,:].shape, emb.shape)

emb2 = capa_o2.predict(opt['B_train'])
print(emb2[:,0,0,:].shape)

emb3 = capa_o2.predict(synthetic_images_B_CTHEALTHY)
print(emb3[:,0,0,:].shape)

emb4 = capa_o2.predict(synthetic_images_B_CTFOSCAL)
print(emb4[:,0,0,:].shape)

## Chi^2 metric

In [ ]:
import pandas as pd
import seaborn as sns
import os

d = pd.read_csv("data/xxx2.csv")

#mb_list = []
mb_synth_ns = []
mb_synth_stroke = []

for i in range(len(opt['trainA_image_names'])):
    #mb_list.append(list(d.loc[d["CT"] == opt['trainA_image_names'][i]]["maskbin"])[0])
    if (list(d.loc[d["CT"] == opt['trainA_image_names'][i]]["maskbin"])[0]==0):
        mb_synth_ns.append(emb2[i,0,0,:])
    else:
        mb_synth_stroke.append(emb2[i,0,0,:])

mb_synth_ns = np.array(mb_synth_ns)
mb_synth_stroke = np.array(mb_synth_stroke)

print(mb_synth_ns.shape)
print(mb_synth_stroke.shape)

In [ ]:
from scipy.stats import chisquare

a = chisquare(mb_synth_ns, mb_synth_stroke[:196])
print(a)

<br><br><br>

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
comp = pca.fit_transform(emb[:,0,0,:])

pca2 = PCA(n_components=2)
comp2 = pca2.fit_transform(emb2[:,0,0,:])

pca3 = PCA(n_components=2)
comp3 = pca3.fit_transform(emb3[:,0,0,:])

pca4 = PCA(n_components=2)
comp4 = pca4.fit_transform(emb4[:,0,0,:])

In [ ]:
plt.scatter(comp[:,0], comp[:,1], label="Synthetic MRI", alpha=0.5)
plt.scatter(comp2[:,0], comp2[:,1], label="MRI_train", alpha=0.3)
plt.scatter(comp3[:,0], comp3[:,1], label="Synthetic MRI CTHEALTHY", alpha=0.1)
plt.scatter(comp4[:,0], comp4[:,1], label="Synthetic MRI CTFOSCAL", alpha=0.1)
plt.legend()
plt.show()

In [ ]:
import umap

print(emb.shape)
print(emb2.shape)
um = umap.UMAP().fit_transform(emb[:,0,0,:])
um2 = umap.UMAP().fit_transform(emb2[:,0,0,:])
um3 = umap.UMAP().fit_transform(emb3[:,0,0,:])
um4 = umap.UMAP().fit_transform(emb4[:,0,0,:])

plt.scatter(um[:,0], um[:,1], label="Synthetic MRI", alpha=0.5)
plt.scatter(um2[:,0], um2[:,1], label="MRI_train", alpha=0.5)
plt.scatter(um3[:,0], um3[:,1], label="Synthetic MRI CTHEALTHY", alpha=0.5)
plt.scatter(um4[:,0], um4[:,1], label="Synthetic MRI CTFOSCAL", alpha=0.5)
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import os

d = pd.read_csv("data/xxx2.csv")

print(opt['trainA_image_names'][0])

mb_list = []
mb_synth_ns = []
mb_synth_stroke = []

print(type(comp))
for i in range(len(opt['trainA_image_names'])):
    #fname, comp[i,0], comp[i,1], maskbin
    mb_list.append(list(d.loc[d["CT"] == opt['trainA_image_names'][i]]["maskbin"])[0])
    if (list(d.loc[d["CT"] == opt['trainA_image_names'][i]]["maskbin"])[0]==0):
        mb_synth_ns.append([comp[i,0], comp[i,1]])
    else:
        mb_synth_stroke.append([comp[i,0], comp[i,1]])

mb_synth_ns = np.array(mb_synth_ns)
mb_synth_stroke = np.array(mb_synth_stroke)


#CENTROIDES
c_stroke = np.mean(mb_synth_stroke, axis=0)
c_ns = np.mean(mb_synth_ns, axis=0)
c_h = np.mean(comp3, axis=0)
print(c_stroke)
print(c_ns)
print(c_h)
#for i in mb_synth_stroke.shape[0]:
#    mb_synth_stroke[:,0]


#d = {'files': opt["trainA_image_names"], 'maskbin': mb_list, 'x_umap': um[:,0], 'y_umap': um[:,1], 'x_pca': comp[:,0], 'y_pca': comp[:,1]}
d = {'files': opt["trainA_image_names"], 'maskbin': mb_list, 'x_pca': comp[:,0], 'y_pca': comp[:,1]}
d = pd.DataFrame(data=d)

#sns.set(rc={'figure.figsize':(8,5)})
#sns.scatterplot(data=d, x="x_pca", y="y_pca", hue="maskbin", alpha=0.9)
plt.figure(figsize=(10,6))

plt.scatter(mb_synth_stroke[:,0], mb_synth_stroke[:,1], label="1 (stroke) Synthetic", alpha=0.5, s=20, marker=".", color="#FF6600")
plt.scatter(mb_synth_ns[:,0], mb_synth_ns[:,1], label="0 (negative) Synthetic", alpha=0.7, s=20, marker=".", color="#0000FF")
plt.scatter(comp3[:,0], comp3[:,1], label="Synthetic MRI CTHEALTHY", alpha=0.1, s=20, marker=".", color="#00AA00")
plt.scatter(c_stroke[0], c_stroke[1], label="", alpha=1, s=100, marker="x", color="#FF3300")
plt.scatter(c_ns[0], c_ns[1], label="", alpha=1, s=100, marker="x", color="#0000FF")
plt.scatter(c_h[0], c_h[1], label="", alpha=1, s=100, marker="x", color="#146d01")
#plt.scatter(comp4[:,0], comp4[:,1], label="Synthetic MRI CTFOSCAL", alpha=0.8, s=5, color="#FF0000")
#plt.xticks(np.arange(-3, 20, 1))
plt.legend(framealpha=0.5)
plt.show()

In [ ]:
for k in range(d.shape[0]):
    if(d.iloc[k]["x_pca"]>-2 and d.iloc[k]["x_pca"]<-1 and d.iloc[k]["y_pca"]>0 and d.iloc[k]["y_pca"]<2
       and d.iloc[k]["maskbin"]==1):
        print(d.iloc[k]["maskbin"], d.iloc[k]["x_pca"], d.iloc[k]["y_pca"], d.iloc[k]["files"])


### UMAP

In [ ]:
import umap

print(emb.shape)
print(emb2.shape)
um = umap.UMAP().fit_transform(emb[:,0,0,:])
um2 = umap.UMAP().fit_transform(emb2[:,0,0,:])
um3 = umap.UMAP().fit_transform(emb3[:,0,0,:])
um4 = umap.UMAP().fit_transform(emb4[:,0,0,:])

plt.scatter(um[:,0], um[:,1], label="Synthetic MRI", alpha=0.5)
plt.scatter(um2[:,0], um2[:,1], label="MRI_train", alpha=0.5)
plt.scatter(um3[:,0], um3[:,1], label="Synthetic MRI CTHEALTHY", alpha=0.5)
plt.scatter(um4[:,0], um4[:,1], label="Synthetic MRI CTFOSCAL", alpha=0.5)
plt.legend()
plt.show()

In [ ]:
import pandas as pd
import seaborn as sns
import os

d = pd.read_csv("data/xxx2.csv")

print(opt['trainA_image_names'][0])

mb_list = []

for i in range(len(opt['trainA_image_names'])):
    #fname, comp[i,0], comp[i,1], maskbin
    mb_list.append(list(d.loc[d["CT"] == opt['trainA_image_names'][i]]["maskbin"])[0])

d = {'files': opt["trainA_image_names"], 'maskbin': mb_list, 'x_umap': um[:,0], 'y_umap': um[:,1], 'x_pca': comp[:,0], 'y_pca': comp[:,1]}
d = pd.DataFrame(data=d)
#d.head()
#d.to_csv('pd-maskbin-'+str(ep)+'epoch.csv')

#sns.scatterplot(data=d, x="x_pca", y="y_pca", hue="maskbin")
sns.set(rc={'figure.figsize':(8,5)})
sns.scatterplot(data=d, x="x_umap", y="y_umap", hue="maskbin", alpha=0.6)


<br><br><br><br>

# MRI (trainB)

In [ ]:
import pandas as pd
import seaborn as sns
import os

d = pd.read_csv("data/listas-mri-maskbin.csv")
print("shape: ",d.shape)
print(len(opt['trainB_image_names']))
print(opt['trainB_image_names'][0])

mb_list = []

mb_mri_ns = []
mb_mri_stroke = []

for i in range(len(opt['trainB_image_names'])):
    #fname, comp[i,0], comp[i,1], maskbin
    mb_list.append(list(d.loc[d["MRI"] == opt['trainB_image_names'][i]]["maskbin"])[0])
    if (list(d.loc[d["MRI"] == opt['trainB_image_names'][i]]["maskbin"])[0]==0):
        mb_mri_ns.append([comp2[i,0], comp2[i,1]])
    else:
        mb_mri_stroke.append([comp2[i,0], comp2[i,1]])

mb_mri_ns = np.array(mb_mri_ns)
mb_mri_stroke = np.array(mb_mri_stroke)

d = {'files': opt["trainB_image_names"], 'maskbin': mb_list, 'x_umap': um2[:,0], 'y_umap': um2[:,1], 'x_pca': comp2[:,0], 'y_pca': comp2[:,1]}
d = pd.DataFrame(data=d)
#d.head()
#d.to_csv('pd-maskbin-'+str(ep)+'epoch.csv')

#sns.scatterplot(data=d, x="x_pca", y="y_pca", hue="maskbin")
#sns.set(rc={'figure.figsize':(8,5)})
#sns.scatterplot(data=d, x="x_pca", y="y_pca", hue="maskbin", alpha=0.9)

plt.scatter(mb_mri_ns[:,0], mb_mri_ns[:,1], label="0 (negative) MRI", alpha=0.5)
plt.scatter(mb_mri_stroke[:,0], mb_mri_stroke[:,1], label="1 (stroke) MRI", alpha=0.5)
plt.scatter(comp3[:,0], comp3[:,1], label="Synthetic MRI CTHEALTHY", alpha=0.5)
plt.legend()
plt.show()


### UMAP

In [ ]:
import pandas as pd
import seaborn as sns
import os

d = pd.read_csv("data/listas-mri-maskbin.csv")
print("shape: ",d.shape)
print(len(opt['trainB_image_names']))
print(opt['trainB_image_names'][0])

mb_list = []

for i in range(len(opt['trainB_image_names'])):
    #fname, comp[i,0], comp[i,1], maskbin
    mb_list.append(list(d.loc[d["MRI"] == opt['trainB_image_names'][i]]["maskbin"])[0])

d = {'files': opt["trainB_image_names"], 'maskbin': mb_list, 'x_umap': um2[:,0], 'y_umap': um2[:,1], 'x_pca': comp2[:,0], 'y_pca': comp2[:,1]}
d = pd.DataFrame(data=d)
#d.head()
#d.to_csv('pd-maskbin-'+str(ep)+'epoch.csv')

#sns.scatterplot(data=d, x="x_pca", y="y_pca", hue="maskbin")
sns.set(rc={'figure.figsize':(8,5)})
sns.scatterplot(data=d, x="x_umap", y="y_umap", hue="maskbin", alpha=0.9)
